# Churn Model Prediction

## Import the libraries needed

In [118]:
# ignore local warnings
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
sns.set(style="darkgrid")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

# Model packages
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from keras.wrappers.scikit_learn import KerasClassifier

## Import the dataset

In [119]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Exploratory Data Analysis (EDA)

In [120]:
df.shape

(10000, 14)

In [121]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


We can see that there are no missing data in the dataset. We will proceed with exploring each of the feature and try to find the best approach to each of the features.

We won't need to include the first three columns in our model since "CustomerId", "Surname" and "RowNumber" do not  have any logical contribution to our prediction.

## Data Pre-processing

In [123]:
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

#### Categorical Variable Encoding

We will encode two categorical variables: "Geography" and "Gender".
The way to encode feature "Geography" is different from "Gender" as we only have two possible values for "Gender" (Female or Male). We can simply encode one of the to be "1" and the other one to be "0". But in "Geography" case, we have more than two possible values. If we encode this feature the same way as "Gender" there will be some numerical order in the value of this feature which would create some confusion in our model (we do not want to create a correlation between the numerical order in the feature and our target variable) 

In [124]:
# Gender
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [125]:
# Geography
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])] , remainder = "passthrough")
X = np.array(ct.fit_transform(X))

#### Split data

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

#### Feature Scaling

We will need to scale some of the features (some numerical features). The reason we are doing this is because we do not want one feature to overweight the other because of their relatively bigger range number. We will use methods of standardization to scale some of the numerical values.

Here's the formula for standardization:

#### $ x_{standardized} = \frac{x - \mu(X)}{\sigma(X)} $

In [127]:
scaler = StandardScaler()
X_train[:, [3,5,6,7,8,11]] = scaler.fit_transform(X_train[:, [3,5,6,7,8,11]])
X_test[:, [3,5,6,7,8,11]] = scaler.transform(X_test[:, [3,5,6,7,8,11]])

## Applying different classification models
### 1) XGBoost Classifier

In [128]:
classifier_1 = XGBClassifier()
classifier_1.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=0, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       validate_parameters=1, verbosity=None)

#### Confusion Matrix

In [129]:
y_pred_1 = classifier_1.predict(X_test)
print(confusion_matrix(y_test,y_pred_1))

[[745  49]
 [101 105]]


#### K-fold Cross Validation

In [130]:
acc_1 = cross_val_score(estimator = classifier_1, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(acc_1.mean()*100))
print("Standard Deviation: {:.2f} %".format(acc_1.std()*100))

Accuracy: 85.41 %
Standard Deviation: 0.71 %


### 2) Random Forest Classifier

In [131]:
classifier_2 = RandomForestClassifier(n_estimators = 100, random_state = 0)
classifier_2.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

#### Confusion Matrix

In [132]:
y_pred_2 = classifier_2.predict(X_test)
print(confusion_matrix(y_test,y_pred_2))

[[757  37]
 [103 103]]


#### K-fold Cross Validation

In [133]:
acc_2 = cross_val_score(estimator = classifier_2, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(acc_2.mean()*100))
print("Standard Deviation: {:.2f} %".format(acc_2.std()*100))

Accuracy: 86.21 %
Standard Deviation: 0.75 %


### 3) Naive Bayes Classifier

In [134]:
classifier_3 = GaussianNB()
classifier_3.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

#### Confusion Matrix

In [135]:
y_pred_3 = classifier_3.predict(X_test)
print(confusion_matrix(y_test,y_pred_3))

[[734  60]
 [121  85]]


#### K-fold Cross Validation

In [136]:
acc_3 = cross_val_score(estimator = classifier_3, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(acc_3.mean()*100))
print("Standard Deviation: {:.2f} %".format(acc_3.std()*100))

Accuracy: 81.88 %
Standard Deviation: 1.11 %


### 4) Logistic Regression

In [137]:
classifier_4 = LogisticRegression(random_state = 0)
classifier_4.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

#### Confusion Matrix

In [138]:
y_pred_4 = classifier_4.predict(X_test)
print(confusion_matrix(y_test,y_pred_4))

[[761  33]
 [159  47]]


#### K-fold Cross Validation

In [139]:
acc_4 = cross_val_score(estimator = classifier_4, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(acc_4.mean()*100))
print("Standard Deviation: {:.2f} %".format(acc_4.std()*100))

Accuracy: 81.02 %
Standard Deviation: 0.71 %


### 5) K-Nearest Neighbors

In [140]:
classifier_5 = KNeighborsClassifier(n_neighbors = 10, metric= "minkowski", p=2)
classifier_5.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

#### Confusion Matrix

In [141]:
y_pred_5 = classifier_5.predict(X_test)
print(confusion_matrix(y_test,y_pred_5))

[[759  35]
 [129  77]]


#### K-fold Cross Validation

In [142]:
acc_5 = cross_val_score(estimator = classifier_5, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(acc_5.mean()*100))
print("Standard Deviation: {:.2f} %".format(acc_5.std()*100))

Accuracy: 84.63 %
Standard Deviation: 0.69 %


### 6) Artifical Neural Network

#### Initializing

In [143]:
ann = tf.keras.models.Sequential()

#### Adding layers

In [144]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=12)) #first layer
ann.add(tf.keras.layers.Dense(units=6, activation="relu")) #Second layer 
ann.add(tf.keras.layers.Dense(units=6, activation="relu")) #Third layer
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid")) #Output layer

#### Compile

In [145]:
ann.compile(optimizer="adam", loss= "binary_crossentropy", metrics= ["accuracy"])

In [146]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Train on 9000 samples
Epoch 1/100
9000/9000 [==============================] - 0s 49us/sample - loss: 0.5521 - acc: 0.7847
Epoch 2/100
9000/9000 [==============================] - 0s 36us/sample - loss: 0.4561 - acc: 0.7988
Epoch 3/100
9000/9000 [==============================] - 0s 36us/sample - loss: 0.4179 - acc: 0.8191
Epoch 4/100
9000/9000 [==============================] - 0s 36us/sample - loss: 0.4073 - acc: 0.8232
Epoch 5/100
9000/9000 [==============================] - 0s 37us/sample - loss: 0.3993 - acc: 0.8272
Epoch 6/100
9000/9000 [==============================] - 0s 39us/sample - loss: 0.3916 - acc: 0.8284
Epoch 7/100
9000/9000 [==============================] - 0s 38us/sample - loss: 0.3829 - acc: 0.8381
Epoch 8/100
9000/9000 [==============================] - 0s 36us/sample - loss: 0.3740 - acc: 0.8436
Epoch 9/100
9000/9000 [==============================] - 0s 36us/sample - loss: 0.3659 - acc: 0.8504
Epoch 10/100
9000/9000 [==============================] - 0s 35us/sam

#### Confusion Matrix

In [147]:
y_pred_6 = ann.predict(X_test)
y_pred_6 = (y_pred_6 > 0.5)
print(confusion_matrix(y_test, y_pred_6))

[[768  26]
 [114  92]]


In [148]:
acc_6 = accuracy_score(y_test, y_pred_6)
print("Accuracy: {:.2f}%".format(acc_6*100))

Accuracy: 86.00%


There are so many improvement that could be made such as tuning the hyperparameter, adding extra layers (for ANN), etc. All in all, the best classification algorithm that we get for now are Artifical Neural Network (ANN) and Random Forest Classifier, with accuracy score of approximately 86%.